# 기계 번역 실습 프로젝트 (한국어 → 영어)

## 프로젝트 목표
- 한국어 문장을 영어로 번역하는 Seq2Seq 기반 모델 구현
- 기본 Seq2Seq 모델과 Attention 적용 모델 총 3가지 모델 구현 및 비교
- 데이터 전처리부터 토크나이저, 임베딩, 모델 설계, 학습, 평가까지 전체 파이프라인 완성

---

## 데이터셋
- train_set.json, valid_set.json 사용
- JSON 구조: {"data": [{"ko": 한국어문장, "mt": 영어번역문}, ...]}
- 대화체, 일상생활 중심 문장으로 구성

__

## 프로젝트 진행 순서
1. 데이터 로드 및 확인
2. 텍스트 전처리 및 토큰화
3. 어휘 사전 구축 및 인덱스 변환
4. 데이터셋 및 DataLoader 구성
5. Seq2Seq 모델 (기본) 구현
6. Attention 모델 구현
7. 모델 학습 및 검증
8. 번역 결과 테스트 및 비교 분석
9. BLEU 등 정량 평가

In [23]:
# @title 환경설정 및 라이브러리 설치
import json
import os
import random
from pathlib import Path
from collections import Counter
from typing import List, Tuple
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import torch.optim as optim


In [3]:
# @title 기본설정

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Device : {device}')

Device : cpu


# 1. 데이터 로드 및 확인

In [6]:
# @title 파일 경로

TRAIN_PATH = 'koen_train_set.json'
VALID_PATH = 'koen_valid_set.json'


In [7]:
def load_json_data(path):
  with open(path, 'r', encoding='utf-8') as f:
    data_json = json.load(f)
  data = data_json['data']
  # 한국어 문장, 영어 문장 추출
  ko_sentences = [item['ko'] for item in data]
  en_sentences = [item['mt'] for item in data]
  return ko_sentences, en_sentences

train_ko, train_en = load_json_data(TRAIN_PATH)
valid_ko, valid_en = load_json_data(VALID_PATH)


print(f"훈련 문장 수: {len(train_ko)}")
print(f"검증 문장 수: {len(valid_ko)}")

print("훈련 샘플 예시:")
print("한국어:", train_ko[0])
print("영어:", train_en[0])

훈련 문장 수: 1200000
검증 문장 수: 150000
훈련 샘플 예시:
한국어: 원하시는 색상을 회신해 주시면 바로 제작 들어가겠습니다.
영어: If you reply to the color you want, we will start making it right away.


# 2. 텍스트 전처리 및 토큰화

In [8]:
from collections import Counter
import re

# 띄어쓰기 기준
def tokenize(sentence):
  # 소문자 변환 및 특수문자 제거
  sentence = sentence.lower()
  sentence = re.sub(r"[^a-zA-Z가-힣0-9\s]", "", sentence)
  tokens = sentence.strip().split()
  return tokens

# 토큰화된 문장 리스트 생성
train_ko_tokens = [tokenize(sentence) for sentence in train_ko]
train_en_tokens = [tokenize(sentence) for sentence in train_en]

print('토큰화 예시:')
print(train_ko_tokens[0])
print(train_en_tokens[0])


토큰화 예시:
['원하시는', '색상을', '회신해', '주시면', '바로', '제작', '들어가겠습니다']
['if', 'you', 'reply', 'to', 'the', 'color', 'you', 'want', 'we', 'will', 'start', 'making', 'it', 'right', 'away']


# 3. 어휘 사전 구축 및 인덱스 변환

In [10]:
# 특수 토큰 정의
PAD_TOKEN = "<PAD>" # Padding Token: 문장 길이를 모두 같게 맞출 때 빈자리 트콘
SOS_TOKEN = "<SOS>" # Start Of Sentence: 문장의 시작을 알리는 토큰
EOS_TOKEN = "<EOS>" # End Of Sentence: 문장의 끝을 알리는 토큰
UNK_TOKEN = "<UNK>" # Unknown Token: 어휘 사전에 없는 단어가 등장했을 때 대체하는 토큰

special_tokens = [PAD_TOKEN, SOS_TOKEN, EOS_TOKEN, UNK_TOKEN]

In [11]:
def build_vocab(tokenized_sentences, min_freq=2):
  counter = Counter()
  for sentence in tokenized_sentences:
    counter.update(sentence)
  
  # 최소 등장 빈도 이상 단어만 포함
  vocab = [ word for word, freq in counter.items() if freq >= min_freq ]
  
  # 특수 토큰 앞에 추가
  vocab = special_tokens + vocab
  
  # 단어 -> 인덱스 사전
  word2idx = {word: idx for idx, word in enumerate(vocab)}
  idx2word = {idx: word for word, idx in word2idx.items()} 
  
  return word2idx, idx2word


ko_word2idx, ko_idx2word = build_vocab (train_ko_tokens)
en_word2idx, en_idx2word = build_vocab (train_en_tokens)

print(f'한국어 단어 집합 크기: {len(ko_word2idx)}')
print(f'영어 단어 집합 크기: {len(en_word2idx)}')


한국어 단어 집합 크기: 238393
영어 단어 집합 크기: 41153


In [12]:
# @title 문장 -> 인덱스 시퀀스 변환 (SOS, EOS 추가)

def sentence_to_indices(sentence_tokens, word2idx):
  indices = [word2idx.get(token, word2idx[UNK_TOKEN]) for token in sentence_tokens]
  return [word2idx[SOS_TOKEN]] + indices + [word2idx[EOS_TOKEN]]

train_ko_indices = [sentence_to_indices(sentence, ko_word2idx) for sentence in train_ko_tokens]
train_en_indices = [sentence_to_indices(sentence, en_word2idx) for sentence in train_en_tokens]

print('예시:')
print(train_ko_indices[0])
print(train_en_indices[0])


예시:
[1, 4, 5, 6, 7, 8, 9, 10, 2]
[1, 4, 5, 6, 7, 8, 9, 5, 10, 11, 12, 13, 14, 15, 16, 17, 2]


# 4. 데이터셋 및 DataLoader 구성

In [17]:
# 토큰 길이 리스트 만들기
ko_lengths = [len(tokenize(sent)) for sent in train_ko]

# 평균 길이 계산 (소수점 이하 버림)
avg_length = int(np.mean(ko_lengths))
max_length = int(np.max(ko_lengths))


print(f"한국어 문장 토큰 평균 길이: {avg_length}")
print(f"한국어 문장 토큰 최대 길이: {max_length}")



std_length = int(np.std(ko_lengths))
print(f"한국어 문장 토큰 표준편차: {std_length}")

MAX_LENGTH = avg_length + std_length  # 평균 + 표준편차
print(f"추천 MAX_LENGTH: {MAX_LENGTH}")


한국어 문장 토큰 평균 길이: 6
한국어 문장 토큰 최대 길이: 79
한국어 문장 토큰 표준편차: 3
추천 MAX_LENGTH: 9


In [36]:
MAX_LENGTh = 30

def pad_sequence(seq, max_len, pad_idx):
  return seq+[pad_idx] * (max_len - len(seq)) if len(seq) < max_len else seq[:max_len]

class TranslationDataset(Dataset):
  def __init__(self, src_seqs, trg_seqs, src_pad_idx, trg_pad_idx):
    self.src_seqs = [pad_sequence(seq, MAX_LENGTH, src_pad_idx) for seq in src_seqs]  
    self.trg_seqs = [pad_sequence(seq, MAX_LENGTH, trg_pad_idx) for seq in trg_seqs]
    
  def __len__(self):
    return len(self.src_seqs)
  
  def __getitem__(self, idx):
    return torch.tensor(self.src_seqs[idx]), torch.tensor(self.trg_seqs[idx])

In [37]:
train_dataset = TranslationDataset(train_ko_indices, train_en_indices, ko_word2idx[PAD_TOKEN], en_word2idx[PAD_TOKEN])
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# 5. Seq2Seq 모델 (기본 GRU) 구현

In [38]:
class Encoder(nn.Module):
  def __init__(self, input_dim, embedding_dim, hidden_dim):
    super().__init__()
    self.embedding = nn.Embedding(input_dim, embedding_dim)
    self.gru = nn.GRU(embedding_dim, hidden_dim, batch_first=True)
    
  def forward(self, src):
    embedded = self.embedding(src)
    outputs, hidden = self.gru(embedded)
    return outputs, hidden
  
class Decoder(nn.Module):
  def __init__(self, output_dim, embedding_dim, hidden_dim):
    super().__init__()
    self.embedding = nn.Embedding(output_dim, embedding_dim)
    self.gru = nn.GRU(embedding_dim, hidden_dim, batch_first=True)
    self.fc = nn.Linear(hidden_dim, output_dim)
    
  
  def forward(self, input, hidden):
    input = input.unsqueeze(1)  # (batch) -> (batch, 1)
    embedded = self.embedding(input)
    output, hidden = self.gru(embedded)
    pred = self.fc(output.squeeze(1))
    return pred, hidden

class Seq2Seq(nn.Module):
  def __init__(self, encoder, decoder, device):
    super().__init__()
    self.encoder = encoder
    self.decoder = decoder
    self.device = device
    
  def forward(self, src, trg, teacher_forcing_ratio=0.5):
    batch_size = src.size(0)
    trg_len = trg.size(1)
    trg_vocab_size = self.decoder.embedding.num_embeddings
    
    outputs = torch.zeros(batch_size, trg_len, trg_vocab_size).to(device)
    encoder_outputs, hidden = self.encoder(src)
    input = trg[:, 0]
    
    for t in range(1, trg_len):
      output, hidden = self.decoder(input, hidden)
      outputs[:, t] = output
      teacher_force = torch.rand(1).item() < teacher_forcing_ratio
      top1 = output.argmax(1)
      input = trg[:, t] if teacher_force else top1
    
    return outputs

# 6. Attention 모델 구현 (Bahdanau Attention)

# 7. 모델 학습 및 검증

In [39]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

INPUT_DIM = len(ko_word2idx)
OUTPUT_DIM = len(en_word2idx)
EMBEDDING_DIM = 256
HIDDEN_DIM = 512

In [40]:
encoder = Encoder(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM)
decoder = Decoder(OUTPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM)
basic_model = Seq2Seq(encoder, decoder, device).to(device)

optimizer = optim.Adam(basic_model.parameters())
criterion = nn.CrossEntropyLoss(ignore_index=en_word2idx[PAD_TOKEN])

In [41]:
def train_epoch(model, dataloader, optimizer, criterion, device):
  model.train()
  epoch_loss = 0
  for src, trg in dataloader:
    src, trg = src.to(device), trg.to(device)
    optimizer.zero_grad()
    output = model(src, trg)
    output_dim = output.shape[-1]
    output = output[:, 1:].reshape(-1, output_dim)
    trg = trg[:, 1:].reshape(-1)
    
    loss = criterion(output, trg)
    loss.backward()
    optimizer.step()
    epoch_loss += loss.item()
    
  return epoch_loss / len(dataloader)

In [ ]:
# 학습
for epoch in range(10):
  loss = train_epoch(basic_model, train_loader, optimizer, criterion, device)
  print(f"[Epoch {epoch+1}] LOSS: {loss:.4f}")